# Simple E-VRP implementation


In [1]:
import sys

import numpy as np
import pandas as pd
import random

import res.EV_utilities


To create random matrices, set boolean variables to True according to the cost kind needed.


In [5]:
createTimeMatrix = False
createInfoFile = True

if createTimeMatrix:
    networkSize = 5
    lowVal = 4.0
    upVal = 15.0
    res.EV_utilities.randomMatrixWithZeroDiagonal(networkSize, lower=lowVal, upper=upVal,
                                                  name='timeMatrixRandom',
                                                  indexName='TT(MIN)')

if createInfoFile:
    networkSize = 5
    
    depotNodes = [0]
    idNodes = range(1, networkSize+1)
    customerNodes = random.sample(idNodes, random.randint(1, networkSize-1))
    csNodes = [x for x in idNodes if x not in customerNodes]
    
    info = {}
    
    for i in depotNodes:
        info[i] = {'TYPE':'DEPOT', 'POS_X':0.0, 'POS_Y':0.0}
        
    for i in customerNodes:
        info[i] = {'TYPE':'CUSTOMER', 'POS_X':np.random.uniform(0.0, 150.0),
                   'POS_Y':np.random.uniform(0.0, 150.0)}
        
    for i in csNodes:
        info[i] = {'TYPE':'CS', 'POS_X':np.random.uniform(0.0, 150.0),
                   'POS_Y':np.random.uniform(0.0, 150.0)}
    
    res.EV_utilities.saveInfoMatrix(info, name='timeMatrixRandom')
    

KeyboardInterrupt: 

Import time matrix and show a travel time example


In [8]:
fileName = 'timeMatrixRandom'+str(5)

path = '../data/simpleImplementation/'+str(fileName)+'.csv'
timeMatrix = pd.read_csv(path).set_index("TT(MIN)")
print(timeMatrix,"\n")

# Indexing example
t = timeMatrix.iat[1,2]
print("Travel time from 1 to 2: ", t)



                 0          1          2          3          4
TT(MIN)                                                       
0         0.000000   6.237646   6.212931  14.187236   5.285591
1         7.025874   0.000000   9.529564   9.058473   7.116360
2        11.259912   5.351140   0.000000   7.663575  11.187708
3        13.338912   8.748450  12.305607   0.000000  14.135934
4        14.903217  14.715104  12.068958  10.352232   0.000000 

Travel time from 1 to 2:  9.529563551687088


Import nodes information to differentiate among them


In [14]:
infoMatrix = pd.read_csv("../data/simpleImplementation/infoSymmetric.csv")

depotNodes     = infoMatrix["DEPOT"].dropna().astype("int32").values
customersNodes = infoMatrix["CUSTOMER"].dropna().astype("int32").values
csNodes        = infoMatrix["CS"].dropna().astype("int32").values

print("Depot node: ", depotNodes, "\n")
print("Customers nodes: ", customersNodes, "\n")
print("Charge stations nodes: ", csNodes, "\n")


Depot node:  [0] 

Customers nodes:  [1 3 4] 

Charge stations nodes:  [2] 



Variables setup


In [15]:
nVehicles  = 1

nCustomers      = len(customersNodes)
nChargeStations = len(csNodes)


## Cost function


In [16]:
def costFunctionTime(pathToFollow, tMatrix):
    n = len(pathToFollow)
    arcCosts = np.empty((n-1,1))
    for i in range(0,n-1):
        arcCosts[i] = tMatrix.iat[path[i], path[i+1]]
    return np.sum(arcCosts)


Test the cost function with a simple example


In [17]:
path = [0,3,4,0]   # 22+25+18 = 65
print("Path to be followed: ", path)
print("Cost of path: ", costFunctionTime(path, timeMatrix))

Path to be followed:  [0, 3, 4, 0]
Cost of path:  65.0


In [22]:
# Optimization